In [2]:
# coding: utf-8
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from nltk import word_tokenize
from nltk.tokenize import wordpunct_tokenize

import os
import random

In [3]:
negative = [f for f in os.listdir('./review_polarity 2/txt_sentoken/neg') if f.endswith('.txt')]

In [4]:
positive = [f for f in os.listdir('./review_polarity 2/txt_sentoken/pos') if f.endswith('.txt')]

In [5]:
arr_reviews = []

for review in negative:
    with open('./review_polarity 2/txt_sentoken/neg/' + review, 'r', encoding='utf-8') as r:
        dicti = {}
        text = r.read()
        dicti['class'] = '0'
        dicti['text'] = text
        arr_reviews.append(dicti)
        
for review in positive:
    with open('./review_polarity 2/txt_sentoken/pos/' + review, 'r', encoding='utf-8') as r:
        dicti = {}
        text = r.read()
        dicti['class'] = '1'
        dicti['text'] = text
        arr_reviews.append(dicti)

        
random.shuffle(arr_reviews)  # перемешиваем данные
    
df = pd.DataFrame(arr_reviews)  # создаем датасет


In [6]:
df.head()

,class,text
0,1,the word 'rest' in the title should be stresse...
1,0,"everyone knows that old rule , 'never work wit..."
2,0,capsule : john the baptist is sent from heaven...
3,1,us critic-type people are always shaking our h...
4,1,"you don't have to know poker to like "" rounder..."


In [7]:
df.describe()

,class,text
count,2000,2000
unique,2,2000
top,1,"on june 30 , 1960 , a self-taught , idealistic..."
freq,1000,1


In [8]:
df.groupby('class').describe()

text
class                                                          
0     count                                                1000
      unique                                               1000
      top     how do films like mouse hunt get into theatres...
      freq                                                    1
1     count                                                1000
      unique                                               1000
      top     warren beatty's " bulworth " is a caustic poli...
      freq                                                    1

In [9]:
#  Делим выборку в соотновении 80:20
rev_train, rev_test, class_train, class_test = train_test_split(df['text'], df['class'], test_size=0.2)

In [10]:
rev_train.shape, rev_test.shape, class_train.shape, class_test.shape

((1600,), (400,), (1600,), (400,))

In [11]:
# Векторизация Bag of Words
bow = CountVectorizer()
bow.fit_transform(rev_train)
bowed_rev_train = bow.transform(rev_train)  # тренировочные тексты
bowed_rev_test = bow.transform(rev_test)   # тестовые тексты

In [12]:
# Обучение DummyClassifier
clf = DummyClassifier(strategy='most_frequent', random_state=0)
clf = clf.fit(bowed_rev_train, class_train)

# Вывод результатов по Dummy Classifier
print(classification_report(class_test, clf.predict(bowed_rev_test)))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       205
          1       0.49      1.00      0.66       195

avg / total       0.24      0.49      0.32       400



/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
# Обучаем модель логистической регрессии
clf = LogisticRegression(class_weight='balanced')
clf.fit(bowed_rev_train, class_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [14]:
print(classification_report(class_test, clf.predict(bowed_rev_test)))

             precision    recall  f1-score   support

          0       0.83      0.82      0.83       205
          1       0.82      0.82      0.82       195

avg / total       0.82      0.82      0.82       400



In [15]:
# Векторизация TF-IDF
vectorizer = TfidfVectorizer(min_df=5,
                             max_df = 0.8,
                             sublinear_tf=True,
                             use_idf=True)
tfidf_rev_train = vectorizer.fit_transform(rev_train)
tfidf_rev_test = vectorizer.transform(rev_test)

In [16]:
# Обучение DummyClassifier
clf = DummyClassifier(strategy='most_frequent', random_state=0)
clf = clf.fit(tfidf_rev_train, class_train)

# Вывод результатов по Dummy Classifier
print(classification_report(class_test, clf.predict(tfidf_rev_test)))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       205
          1       0.49      1.00      0.66       195

avg / total       0.24      0.49      0.32       400



/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Наша модель справилась с определением сентимента лучше, чем DummyClassifier

In [17]:
# Обучаем модель логистической регрессии
clf = LogisticRegression(class_weight='balanced')
clf.fit(tfidf_rev_train, class_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [45]:
# Печатаем результаты классификации
print(classification_report(class_test, clf.predict(tfidf_rev_test)))

             precision    recall  f1-score   support

          0       0.85      0.84      0.85       205
          1       0.83      0.85      0.84       195

avg / total       0.84      0.84      0.84       400



Наша модель справилась с определением сентимента лучше, чем DummyClassifier

Линейная регрессия

y=β0+β1x

Логистическая регрессия

log(p1−p)=β0+β1x

In [19]:
clf.coef_

array([[-0.08260499,  0.06580075,  0.04050672, ..., -0.06186723,
         0.01306732,  0.02060943]])

In [20]:
clf.intercept_

array([-0.10536886])

In [21]:
clf.classes_

array(['0', '1'], dtype=object)

In [48]:
logodds = clf.intercept_ + clf.coef_[0] * 2  # compute predicted log-odds for al=2 using the equation
odds = np.exp(logodds)  # convert log-odds to odds
prob = odds/(1 + odds)  # convert odds to probability

In [ ]:
# compute predicted probability for al=2 using the predict_proba method
clf.predict_proba(tfidf_text)[:, 1]

In [ ]:
# examine the coefficient for al
zip(feature_cols, logreg.coef_[0])

In [40]:
# increasing al by 1 (so that al=3) increases the log-odds by 4.18
logodds = 0.64722323 + 4.1804038614510901
odds = np.exp(logodds)
prob = odds/(1 + odds)

0.99205808391674566

In [ ]:
# compute predicted probability for al=3 using the predict_proba method
clf.predict_proba(3)[:, 1]

In [43]:
# examine the intercept
clf.intercept_

array([-0.10536886])

In [50]:
# convert log-odds to probability
logodds = clf.intercept_
odds = np.exp(logodds)
prob = odds/(1 + odds)